In [59]:
# imports
from PyPDF2 import PdfReader

# local do arquivo
file_path = 'RHPP05LA.PDF'

# correcao de dados
SUBSTITUICOES = {
    "1/12 FERIAS"           : "ferias",
    "1/12 DE 1 /3 FERIAS"   : "13_ferias",
    "TOTAL FERIAS"          : "total_ferias",
    "1/12 13 SAL ARIO"      : "13_salario",
    "T O T A L"             : "total",
    "PROVISIONAMENTO"       : "apropriacao",
    "INSS  20,00%"          : "inss",
    "SP-PREVCOM"            : "sp-prevcom",
    "AC TRAB  "             : "ac_trab_",
    "FGTS 8%"               : "fgts",
    "TOTAL DE ENCARGOS"     : "total_encargos",
    "."                     : "",
    ","                     : ".",
    "1 /3 FERIAS"           : "13_ferias", 
    "13 SAL ARIO"           : "13_salario", 
    "SPPREV"                : "spprev"
}

# funcao que divide uma lista em duas listas
# de acordo com a palavra enviada e os pads definidos
def split_tables(
        tb: list, 
        split_word:str="", 
        pad_top_tb1:int=0,
        pad_bottom_tb1:int=0,
        pad_top_tb2:int=0,
        pad_bottom_tb2:int=0
        ):
    
    for i in range(len(tb)):
        if split_word in tb[i]:
            tb1 = tb[pad_top_tb1:i+pad_bottom_tb1]
            tb2  = tb[i+pad_top_tb2:] if pad_bottom_tb2==0 else tb[i+pad_top_tb2:pad_bottom_tb2] 
    return tb1, tb2

# funcao que normatiza palavras-campos das tabelas
def correct_words(tb: list):
    for i in range(len(tb)):
        for search, type in SUBSTITUICOES.items():
            tb[i] = tb[i].replace(search, type)
    return tb

# funcao que coloca a tabela em formato de tupla
# formato tabela: 
## linha  0: cabecalhos
## coluna 0: tipos
## coluna 1 a 5: dados
def cast_table_to_tuple(tb:list) -> tuple:
    headers = tb[0].split()

    datas = tb[1:]
    for i in range(len(datas)):
        datas[i] = datas[i].split()
    
    types = [data[0] for data in datas]

    # correcao bug: separando o valor 13_salario com um espaco
    datas = [data[1:] for data in datas]
    for data in datas:
        if len(data) > 5:
            data[3] = data[3]+data[4]
            data[4] = data[5]
    
    tup = {}
    for type, data in zip(types, datas):
        tup[type] = {}
        for header, value in zip(headers, data):
            tup[type][header] = float(value)

    return tup
    
    

#ler pdf
with open(file_path, 'rb') as pdf_file:
    pdf_reader = PdfReader(pdf_file)
    pages = pdf_reader.pages

    #para cada page indexada com um number_page
    #extrair texto e colocar em lines (retirando as em branco)
    for number_page, page in enumerate(pages, start=1):
        text = page.extract_text()
        lines = text.split('\n')
        lines = [line for line in lines if not line.isspace()]

        # extrai dados da unidade
        unidade = lines[3].strip().split(" - ")
        codigo_unidade = int(unidade[0].replace(" ", ""))
        nome_unidade = unidade[1]

        # se a pagina for impar, os dados sao referentes apropriado e realizado da unidade
        # se a pagina for par, os dados sao referentes aos totais da unidade
        if(number_page %2 == 1):

            # extrai tabela CLT e Autarquico
            tb_clt, tb_aut = split_tables(lines, "A U T A R Q U I C O", 7, -1, 2, -1)

            # extrai apropriado e realizado da tabela
            tb_clt_apropriado, tb_clt_realizado = split_tables(tb_clt, "R E V E R S A O", 1, 0, 1, 0)
            tb_aut_apropriado, tb_aut_realizado = split_tables(tb_aut, "R E V E R S A O", 1, 0, 1)

            # normatiza dados
            tb_clt_apropriado = correct_words(tb_clt_apropriado)
            tb_clt_realizado  = correct_words(tb_clt_realizado)
            tb_aut_apropriado = correct_words(tb_aut_apropriado)
            tb_aut_realizado  = correct_words(tb_aut_realizado)
            
            #converte as listas para tupla
            clt = {} 
            autarquico = {}
            clt['apropriado'] = cast_table_to_tuple(tb_clt_apropriado)
            clt['realizado']  = cast_table_to_tuple(tb_clt_realizado)
            autarquico['apropriado'] = cast_table_to_tuple(tb_aut_apropriado)
            autarquico['realizado']  = cast_table_to_tuple(tb_aut_realizado)

            #display(clt)
            display(autarquico)

        else:
            print('TOTAIS\n\n\n-----')
        

print("end")

{'apropriado': {'apropriacao': {'ferias': 121536.56,
   '13_ferias': 40512.2,
   'total_ferias': 162048.76,
   '13_salario': 121536.56,
   'total': 283585.32},
  'inss': {'ferias': 20141.29,
   '13_ferias': 6713.74,
   'total_ferias': 26855.03,
   '13_salario': 20141.29,
   'total': 46996.32},
  'ac_trab_0.5000%': {'ferias': 503.15,
   '13_ferias': 167.71,
   'total_ferias': 670.86,
   '13_salario': 503.15,
   'total': 1174.01},
  'fgts': {'ferias': 8056.28,
   '13_ferias': 2685.46,
   'total_ferias': 10741.74,
   '13_salario': 8056.28,
   'total': 18798.02},
  'sp-prevcom': {'ferias': 0.0,
   '13_ferias': 0.0,
   'total_ferias': 0.0,
   '13_salario': 2244.59,
   'total': 2244.59},
  'total_encargos': {'ferias': 28700.72,
   '13_ferias': 9566.91,
   'total_ferias': 38267.63,
   '13_salario': 30945.31,
   'total': 69212.94},
  'total': {'ferias': 150237.28,
   '13_ferias': 50079.11,
   'total_ferias': 200316.39,
   '13_salario': 152481.87,
   'total': 352798.26}},
 'realizado': {'apropr

TOTAIS


-----


{'apropriado': {'apropriacao': {'ferias': 158930.39,
   '13_ferias': 52976.78,
   'total_ferias': 211907.17,
   '13_salario': 158930.39,
   'total': 370837.56},
  'inss': {'ferias': 24058.39,
   '13_ferias': 8019.41,
   'total_ferias': 32077.8,
   '13_salario': 24058.39,
   'total': 56136.19},
  'ac_trab_0.5000%': {'ferias': 601.01,
   '13_ferias': 200.36,
   'total_ferias': 801.37,
   '13_salario': 601.01,
   'total': 1402.38},
  'fgts': {'ferias': 9623.05,
   '13_ferias': 3207.7,
   'total_ferias': 12830.75,
   '13_salario': 9623.05,
   'total': 22453.8},
  'sp-prevcom': {'ferias': 0.0,
   '13_ferias': 0.0,
   'total_ferias': 0.0,
   '13_salario': 2946.95,
   'total': 2946.95},
  'total_encargos': {'ferias': 34282.45,
   '13_ferias': 11427.47,
   'total_ferias': 45709.92,
   '13_salario': 37229.4,
   'total': 82939.32},
  'total': {'ferias': 193212.84,
   '13_ferias': 64404.25,
   'total_ferias': 257617.09,
   '13_salario': 196159.79,
   'total': 453776.88}},
 'realizado': {'apropria

TOTAIS


-----


ValueError: could not convert string to float: '732628.980.00'